# Predict points via RandomForestRegressor by sklearn

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from time import localtime, strftime

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 7)

In [2]:
from sklearn.ensemble import RandomForestRegressor

In [56]:
TEST_SAMPLE_NUMBER = 57092
TRAIN_SAMPLE_NUMBER = 285998

In [21]:
# array1: [sX1; sY1; sX2; sY2; ...]
# array2: [eX1; eY1; eX2; eY2; ...]
# output: [dist((sX1, sY1), (eX1, eY1)), dist((sX1, sY1), (eX1, eY1)),
#          dist((sX2, sY2), (eX2, eY2)), dist((sX2, sY2), (eX2, eY2)), ...]

def distance_for_each_point(array1, array2):
    error_template = "Array lengths should be equal. len(array1): {len1}, len(array2): {len2}"
    assert (len(array1) == len(array2)), error_template.format(len1=len(array1), len2=len(array2))
    
    if array1.ndim == 1:
        length = len(array1)
        # array1[i:i+2] -- point i from first array
        # np.linalg.norm -- calculate distance between points
        distance = np.array([np.linalg.norm(array1[i:i+2] - array2[i:i+2]) for i in range(0,length,2)])
        result = np.array([[d, d] for d in distance]).flatten()
    else:
        result = np.array([distance_for_each_point(array1[i], array2[i]) for i in range(len(array1))])
    
    return result

In [30]:
test_data = pd.read_csv('src/test_data_points.csv', index_col=0)

In [31]:
train_data = pd.read_csv('src/train_data_points.csv', index_col=0)

In [32]:
test_results = test_data[5:10]
test_data = test_data[:5]

train_results = train_data[5:10]
train_data = train_data[:5]

In [15]:
def to_model(df):
    return np.array(df).T

In [ ]:
trees_number = np.power(2, range(4, 11))
result = []

for number_of_trees in trees_number:
    print(str(number_of_trees) + ": " + strftime("%Y-%m-%d %H:%M:%S", localtime()))
    est = RandomForestRegressor(n_jobs=-1, n_estimators=number_of_trees)

    est.fit(to_model(train_data), to_model(train_results))
    predicted_results = est.predict(to_model(test_data))
    
    np.save("src/Logs/161003_predicted_coordinates_rfr_trees_num_{}".format(number_of_trees), predicted_results)
    
    result.append(distance_for_each_point(np.array(test_results), predicted_results.T).sum() / TEST_SAMPLE_NUMBER)

16: 2016-10-04 00:13:14
32: 2016-10-04 00:15:35
64: 2016-10-04 00:19:52


In [ ]:
ind = trees_number
width = 0.3
result_bar = plt.bar(ind, result, width, color='g')

plt.ylabel('Average difference')
plt.xlabel('Number of trees')
plt.title('Difference between real points and predicted by number of trees in RFR')
plt.xticks(ind + width/2, ind)
plt.savefig('src/Plots/161004_rfr_difference_by_trees_number.png')